<a href="https://colab.research.google.com/github/eniompw/modded-llama2.c/blob/main/Baby_Llama_128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# training and inference code
!git clone https://github.com/eniompw/modded-llama2.c

Cloning into 'modded-llama2.c'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 4), reused 13 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (21/21), 41.98 KiB | 20.99 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [ ]:
!mkdir -p ./modded-llama2.c/data
# Download files in parallel directly into the target directory
!wget -P ./modded-llama2.c/data https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok128/tok128.model & \
  wget -P ./modded-llama2.c/data https://huggingface.co/datasets/enio/TinyStories/raw/main/tok128/tok128.vocab & \
  wget -P ./modded-llama2.c/data https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok128/tok128.tar.gz & \
  wait

# Untar the large file after all downloads are complete
!tar -I pigz -xf ./modded-llama2.c/data/tok128.tar.gz -C ./modded-llama2.c/data/

# compile run / inference executable
!cd modded-llama2.c && gcc -O3 -o run run.c -lm
# create tok105.bin
!cd modded-llama2.c && python tokenizer.py --tokenizer-model=data/tok128.model

--2025-03-26 21:19:04--  https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok128/tok128.model
Resolving huggingface.co (huggingface.co)... 13.33.45.68, 13.33.45.37, 13.33.45.10, ...
Connecting to huggingface.co (huggingface.co)|13.33.45.68|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/65/c8/65c8615bfd6b6d94c8e62381ee6653578b439f71ff04b86aaefa9828dec8bb70/e4794cc7590440c422b6d63123f6f10fbc7ec83f8fc281c60296cbef61f772ac?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27tok128.model%3B+filename%3D%22tok128.model%22%3B&Expires=1743027545&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MzAyNzU0NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzY1L2M4LzY1Yzg2MTViZmQ2YjZkOTRjOGU2MjM4MWVlNjY1MzU3OGI0MzlmNzFmZjA0Yjg2YWFlZmE5ODI4ZGVjOGJiNzAvZTQ3OTRjYzc1OTA0NDBjNDIyYjZkNjMxMjNmNmYxMGZiYzdlYzgzZjhmYzI4MWM2MDI5NmNiZWY2MWY3NzJhYz9yZXNwb25zZS1jb250ZW50LW

In [9]:
# train neural net
!cd modded-llama2.c && python train.py --vocab_source=custom --vocab_size=128 --compile=False \
  --dim=128 --n_layers=5 --n_heads=8 --n_kv_heads=4 --batch_size=32 \
  --always_save_checkpoint=True --eval_interval=100 --max_iters=100 #--init_from='resume'

Overriding: vocab_source = custom
Overriding: vocab_size = 128
Overriding: compile = False
Overriding: dim = 128
Overriding: n_layers = 5
Overriding: n_heads = 8
Overriding: n_kv_heads = 4
Overriding: batch_size = 32
Overriding: always_save_checkpoint = True
Overriding: eval_interval = 100
Overriding: max_iters = 100
Original vocab size: 128, Padded vocab size: 128
tokens per iteration will be: 65,536
breaks down as: 4 grad accum steps * 1 processes * 32 batch size * 512 max seq len
Initializing a new model from scratch
/content/modded-llama2.c/train.py:195: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(dtype == "float16"))
num decayed parameter tensors: 36, with 937,984 parameters
num non-decayed parameter tensors: 11, with 1,408 parameters
using fused AdamW: True
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
Created a Preto

In [10]:
# inference
!cd ./modded-llama2.c && ./run out/model.bin -z data/tok128.bin -t 0.8 -n 256 -i "Once upon a time "

Once upon a time on a thave ot cape, bunt pily. She dime wel nomiker withat to said or held and gamm bolk. Maned soth nare solf day, " Amom gilys nont, buse funt. Ther all onend hay. <unk>"<unk>I wer cor.<unk>Limet bens a hay cimy. Mamde groked Lid toy and hirs ove shap ugirongen. Onew her ot bily po nene said.<unk>They w
achieved tok/s: 885.416667
